## Simulation of particules in a Box drawn from a particular distribution and their correlation

In [6]:
from lib import pair_correlation, make_bins, trace_pair_correlation

In [7]:
import numpy as np 
import math 
import scipy as sp
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from mpl_toolkits import mplot3d
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from scipy.spatial import cKDTree

In [8]:
n = 100000

### Uniform distribution


#### Sample

In [9]:
data_unif = np.random.uniform(0,1,(n,3))

Create boxes around for periodic distance calculus 

### Gaussian distribution truncated to [0,1]



#### Sample 

In [10]:
lower, upper = 0, 1
mu, sigma = 0.5, 0.2

x_norm = stats.truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=n)
y_norm = stats.truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=n)
z_norm = stats.truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=n)

In [11]:
data_norm = np.stack((x_norm, y_norm, z_norm), axis=-1)

### Log-normal distribution truncated to [0,1]

#### Sample 

In [12]:
# create log-normal from normal by the formula exp(N(0,1)) = log-normal(0,1)
#we want a log normal with values between 0 and 1 
sig = 0.35
s = 0.38

x_log = stats.lognorm.rvs(s, loc=0, scale=sig, size=n)
y_log = stats.lognorm.rvs(s, loc=0, scale=sig, size=n)
z_log = stats.lognorm.rvs(s, loc=0, scale=sig, size=n)
# we need to truncate it to values between 0 and 1 +chose parameter 

In [13]:
data_log = np.stack((x_log, y_log, z_log), axis=-1)
data_log = np.delete( data_log, np.where(data_log > 1) , axis = 0 )

### Correlation function

In [14]:
r_max = 0.5 
dr = 0.01

try to fit to those types of functions after : 


In [15]:
def f_norm(x, mu, sigma):
    return (1/(np.sqrt(2*np.pi)*sigma)*np.exp(-((x-mu)**2)/(2*sigma**2)))

In [16]:
def polynome(x, a, b, c, d, e):
    return (a*(x**d)+b*(x**e)+c)

In [17]:
def polynome_inv(x, a, b, c):
    return (a*(1/x)+b*(1/(x**2))+c)

### Uniform

In [18]:
len(data_unif)

100000

In [19]:
#abscisse, correlation = pair_correlation(data_unif, r_max, dr)

In [20]:
#np.savetxt("Unif_corr_100_000_rmax=0.5_dr=0.01.txt", [abscisse, correlation])

In [21]:
abscisse, correlation = np.loadtxt("Unif_corr_100_000_rmax=0.5_dr=0.01.txt")

In [ ]:
fig = trace_pair_correlation(abscisse[1:], correlation[1:], 'uniform',"PaleGreen", 'green', n)

In [ ]:
fig.show()

In [ ]:
a,b,c, d, e = sp.optimize.curve_fit(polynome, abscisse[1:], correlation[1:], maxfev = 1200000 )

In [ ]:
x_min = 0
x_max = 0.5

mean = 0.5
std = 0.15

x = np.linspace(0, 0.5, 100)

fig.add_trace(go.Scatter(x = np.linspace(0.1, 0.5, 100), y =polynome(x,a, b,c)))

In [ ]:
x = np.arange(0, k*n, n)
y = evol_time
fig = px.scatter(x = x, y = y)
fig.show()
evol_time # en secondes

### Normal

In [ ]:
abscisse_norm, correlation_norm = pair_correlation(data_norm, 0.5, 0.01)

In [ ]:
np.savetxt("Norm_corr_100_000_rmax=0.5_dr=0.01.txt", [abscisse_norm, correlation_norm])

In [ ]:
abscisse_norm, correlation_norm = np.loadtxt("Norm_corr_100_000_rmax=0.5_dr=0.01.txt")

In [ ]:
fig_norm1 = trace_pair_correlation(abscisse_norm, correlation_norm, 'normal','lightblue', 'blue', n)
fig_norm1.show()

In [ ]:
a,b,c  = sp.optimize.curve_fit(polynome, abscisse_norm[1:], correlation_norm[1:] )[0]

In [ ]:
x = np.linspace(0, 0.5, 40)
fig_norm1.add_trace(go.Scatter(x = x, y = polynome(x, a,b,c) , marker_color = 'pink'))

#### Other method (see overleaf)

In [ ]:
corr_norm = make_bins(data_norm, r_max, dr) / make_bins(data_unif, r_max, dr)

In [ ]:
corr_norm = corr_norm - 1 
abs_corr_norm = np.arange(0, r_max, dr)

In [ ]:
#np.savetxt("Norm_corr_autre_methode_100_000_rmax=0.5_dr=0.01.txt", [abs_corr_norm, corr_norm])

In [ ]:
abs_corr_norm, corr_norm = np.loadtxt("Norm_corr_autre_methode_100_000_rmax=0.5_dr=0.01.txt")

In [ ]:
fig_norm_2 = trace_pair_correlation(abs_corr_norm, corr_norm, 'normal', 'lightblue', 'blue', n)

In [ ]:
fig_norm_2

In [ ]:
mu, sig = sp.optimize.curve_fit(f_norm, abs_corr_norm, corr_norm)[0]

In [ ]:
mu

In [ ]:
sig

In [ ]:
x = np.linspace(0, 0.5, 30)
fig_norm_2.add_trace(go.Scatter(x = x, y = f_norm(x,mu,sig),marker_color = 'deepskyblue'))

### Log-normal

In [ ]:
absi_log , corr_log = pair_correlation(data_log, 0.5, 0.01)

In [ ]:
np.savetxt("Log_corr_100_000_rmax=0.5_dr=0.01.txt", [absi_log , corr_log])

In [ ]:
absi_log , corr_log = np.loadtxt("Log_corr_50000_rmax=0.5_dr=0.01.txt")

In [ ]:
fig_log = trace_pair_correlation(absi_log, corr_log, 'log-normal', 'mistyrose', 'red', n)

In [ ]:
fig_log.show()

In [ ]:
a,b,c,d,e = sp.optimize.curve_fit(polynome, absi_log[1:] , corr_log[1:], maxfev = 1200000)[0]

In [ ]:
a

In [ ]:
d

In [ ]:
c

In [ ]:
x = np.linspace(0, 0.5, 30)
fig_log.add_trace(go.Scatter(x = x[1:], y = polynome(x[1:], a, b, c, d, e), marker_color = 'green'))

In [ ]:
a,b,c  = sp.optimize.curve_fit(polynome_inv, absi_log[1:] , corr_log[1:])[0]

In [ ]:
a

In [ ]:
b

In [ ]:
c

In [ ]:
x = np.linspace(0, 0.5, 30)
fig_log.add_trace(go.Scatter(x = x[1:], y = polynome_inv(x[1:], a, b, c), marker_color = 'firebrick'))

## Data

In [22]:
data = np.loadtxt("xz0")
x = data[:,0]
y = data[:,1]
z = data[:,2]

In [23]:
data = np.stack((x, y, z), axis=-1)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(x, y, z, c=z, cmap='Blues');

In [24]:
n_select = 200000
number_of_rows = data.shape[0]
random_indices = np.random.choice(number_of_rows, size=n_select, replace=False)
random_rows = data[random_indices, :]

In [ ]:
correlation_data = pair_correlation(random_rows, r_max, dr)

In [ ]:
fig_data = trace_pair_correlation(correlation_data[0], correlation_data[1], 'unknown','lightblue', 'pink', n_select)
fig_data.show()

In [ ]:
(correlation_data[0], correlation_data[1])

## Bins not corr 

In [ ]:
bins_norm = make_bins(data_norm, r_max, dr)

In [ ]:
# Bins
fig = px.scatter(x = np.arange(0, r_max, dr),y = bins_norm)
fig.update_layout(
    title="Bins for pair correlation function of a normal distribution before normalization",
    xaxis_title="radius",
    yaxis_title="correlation",
    legend_title="",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="RebeccaPurple"
    )
)
fig.show()

In [ ]:
bins_log = make_bins(data_log, r_max, dr)

In [ ]:
# Bins 
fig = px.scatter(x = np.arange(0, r_max, dr),y = bins_log)
fig.update_layout(
    title="Bins for pair correlation function of a log-normal distribution before normalization",
    xaxis_title="radius",
    yaxis_title="correlation",
    legend_title="",
    font=dict(
        family="Courier New, monospace",
        size=14,
        color="RebeccaPurple"
    )
)
fig.show()

## Temporary 

In [ ]:
import time
k = 20
n = 1000
t0 = time.time()
evol_time = np.empty(k)
for i in range(k):
    data_unif = np.random.uniform(0,1,(n*i+1,3))
    correlation = pair_correlation(data_unif, 0.5, 0.01)
    t1 = time.time()
    evol_time[i] = t1-t0
    t0=t1
    
time.time()

In [ ]:
fig = px.scatter(x = np.arange(0,20000, 1000),y = evol_time)

In [ ]:
fig.show()

In [ ]:
%%time
tree = cKDTree(data_norm)
r_max = math.sqrt(2)/2
dr = 0.03
b = np.zeros(n)
bins_norm = np.empty(math.floor(r_max/dr)+1)
for i in range(math.floor(r_max/dr)+1):
    a = tree.query_ball_point(data_norm, i*dr+dr, return_length = True)
    bins_norm[i] = sum(a) - sum(b)
    b = a
sum(bins_norm)